In [1]:
# All necessary libraries must be put here
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import mplfinance as mpf
import plotly.graph_objects as go
import pandas as pd
import cv2
import matplotlib.image as mpimg

In [2]:
# Read the CRSP Data and make it become a dataframe
crsp_data_1993_2019 = pd.read_csv("/Users/namnguyen/Desktop/Research/Don/Imaging_Price_Trends_Code/crsp_data_20240719_1.csv")

# Check the data
crsp_data_1993_2019.head(5)

/var/folders/pl/z1lfm5xs7299k23shpwd6ngh0000gp/T/ipykernel_90371/4244800754.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  crsp_data_1993_2019 = pd.read_csv("/Users/namnguyen/Desktop/Research/Don/Imaging_Price_Trends_Code/crsp_data_20240719_1.csv")


,PERMNO,HdrCUSIP,Ticker,PERMCO,DlyCalDt,DlyVol,DlyClose,DlyLow,DlyHigh,DlyOpen
0,10001,36720410,GFGC,7953,1993-01-04,150.0,14.5,14.50,14.5,14.50
1,10001,36720410,GFGC,7953,1993-01-05,0.0,NaN,NaN,NaN,NaN
2,10001,36720410,GFGC,7953,1993-01-06,0.0,NaN,NaN,NaN,NaN
3,10001,36720410,GFGC,7953,1993-01-07,228.0,14.5,14.50,14.5,14.50
4,10001,36720410,GFGC,7953,1993-01-08,1375.0,14.5,14.25,14.5,14.25


In [6]:
# At this part, we focus first on training sample
# Training data are taken from 1993 to 2000. 

crsp_data_1993_2019["DlyCalDt"] = pd.to_datetime(crsp_data_1993_2019["DlyCalDt"])

# Filter dates from 1993 to 2000
start_date = "1993-01-01"
end_date = "2000-12-31" 
crsp_data_1993_2000 = crsp_data_1993_2019[(crsp_data_1993_2019["DlyCalDt"] >= start_date) & (crsp_data_1993_2019["DlyCalDt"] <= end_date)]

# Check the last 5 values
print(crsp_data_1993_2000.tail(5))

# Check the shape of the crsp_data_1993_2000
print(crsp_data_1993_2000.shape) # (17223740, 10)

# Check the shape of the crsp_data_1993_2019
print(crsp_data_1993_2019.shape) # (51131155, 10)

          PERMNO  HdrCUSIP Ticker  PERMCO   DlyCalDt   DlyVol  DlyClose  \
50923819   93316  98960110   ZING    7469 2000-03-13  10685.0   15.3750   
50923820   93316  98960110   ZING    7469 2000-03-14   1800.0   15.0625   
50923821   93316  98960110   ZING    7469 2000-03-15   2100.0   15.0625   
50923822   93316  98960110   ZING    7469 2000-03-16    700.0   15.3750   
50923823   93316  98960110    NaN    7469 2000-03-17      NaN       NaN   

           DlyLow  DlyHigh  DlyOpen  
50923819  15.1250   15.375  15.2500  
50923820  15.0625   15.375  15.3750  
50923821  15.0625   15.375  15.0625  
50923822  15.0625   15.375  15.0625  
50923823      NaN      NaN      NaN  
(17223740, 10)
(51131155, 10)


In [12]:
# Drop all the NaN

crsp_data_1993_2000 = crsp_data_1993_2000.dropna()

# Check the shape again
print(crsp_data_1993_2000.shape) # (15555109, 10)

(15555109, 10)


In [7]:
# Function to calculate 5-day log returns
def calculate_5_day_log_returns(group):
    # Create a column for the price 5 days later
    group["DlyClose_5_days_later"] = group["DlyClose"].shift(-5)

    # Calculate Log Returns
    group["Log_Return"] = np.log(group["DlyClose_5_days_later"] / group["DlyClose"])

    return group 

In [8]:
# Calculate 5-day stock returns for the whole dataset from 1993 to 2000
crsp_data_1993_2000_R5 = crsp_data_1993_2000.groupby("PERMNO").apply(calculate_5_day_log_returns)

# Display the result
crsp_data_1993_2000_R5

/var/folders/pl/z1lfm5xs7299k23shpwd6ngh0000gp/T/ipykernel_90371/3072626669.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  crsp_data_1993_2000_R5 = crsp_data_1993_2000.groupby("PERMNO").apply(calculate_5_day_log_returns)


PERMNO  HdrCUSIP Ticker  PERMCO   DlyCalDt   DlyVol  \
PERMNO                                                                 
10001  0          10001  36720410   GFGC    7953 1993-01-04    150.0   
       1          10001  36720410   GFGC    7953 1993-01-05      0.0   
       2          10001  36720410   GFGC    7953 1993-01-06      0.0   
       3          10001  36720410   GFGC    7953 1993-01-07    228.0   
       4          10001  36720410   GFGC    7953 1993-01-08   1375.0   
...                 ...       ...    ...     ...        ...      ...   
93316  50923819   93316  98960110   ZING    7469 2000-03-13  10685.0   
       50923820   93316  98960110   ZING    7469 2000-03-14   1800.0   
       50923821   93316  98960110   ZING    7469 2000-03-15   2100.0   
       50923822   93316  98960110   ZING    7469 2000-03-16    700.0   
       50923823   93316  98960110    NaN    7469 2000-03-17      NaN   

                 DlyClose   DlyLow  DlyHigh  DlyOpen  DlyClose_5_days_later  \
PERMNO                                                                        
10001  0          14.5000  14.5000   14.500  14.5000                  14.50   
       1              NaN      NaN      NaN      NaN                    NaN   
       2              NaN      NaN      NaN      NaN                  13.75   
       3          14.5000  14.5000   14.500  14.5000                  13.75   
       4          14.5000  14.2500   14.500  14.2500                    NaN   
...                   ...      ...      ...      ...                    ...   
93316  50923819   15.3750  15.1250   15.375  15.2500                    NaN   
       50923820   15.0625  15.0625   15.375  15.3750                    NaN   
       50923821   15.0625  15.0625   15.375  15.0625                    NaN   
       50923822   15.3750  15.0625   15.375  15.0625                    NaN   
       50923823       NaN      NaN      NaN      NaN                    NaN   

                 Log_Return  
PERMNO                       
10001  0            0.00000  
       1                NaN  
       2                NaN  
       3           -0.05311  
       4                NaN  
...                     ...  
93316  50923819         NaN  
       50923820         NaN  
       50923821         NaN  
       50923822         NaN  
       50923823         NaN  

[17223740 rows x 12 columns]

In [11]:
crsp_data_1993_2000_R5

PERMNO  HdrCUSIP Ticker  PERMCO   DlyCalDt   DlyVol  \
PERMNO                                                                 
10001  0          10001  36720410   GFGC    7953 1993-01-04    150.0   
       1          10001  36720410   GFGC    7953 1993-01-05      0.0   
       2          10001  36720410   GFGC    7953 1993-01-06      0.0   
       3          10001  36720410   GFGC    7953 1993-01-07    228.0   
       4          10001  36720410   GFGC    7953 1993-01-08   1375.0   
...                 ...       ...    ...     ...        ...      ...   
93316  50923819   93316  98960110   ZING    7469 2000-03-13  10685.0   
       50923820   93316  98960110   ZING    7469 2000-03-14   1800.0   
       50923821   93316  98960110   ZING    7469 2000-03-15   2100.0   
       50923822   93316  98960110   ZING    7469 2000-03-16    700.0   
       50923823   93316  98960110    NaN    7469 2000-03-17      NaN   

                 DlyClose   DlyLow  DlyHigh  DlyOpen  DlyClose_5_days_later  \
PERMNO                                                                        
10001  0          14.5000  14.5000   14.500  14.5000                  14.50   
       1              NaN      NaN      NaN      NaN                    NaN   
       2              NaN      NaN      NaN      NaN                  13.75   
       3          14.5000  14.5000   14.500  14.5000                  13.75   
       4          14.5000  14.2500   14.500  14.2500                    NaN   
...                   ...      ...      ...      ...                    ...   
93316  50923819   15.3750  15.1250   15.375  15.2500                    NaN   
       50923820   15.0625  15.0625   15.375  15.3750                    NaN   
       50923821   15.0625  15.0625   15.375  15.0625                    NaN   
       50923822   15.3750  15.0625   15.375  15.0625                    NaN   
       50923823       NaN      NaN      NaN      NaN                    NaN   

                 Log_Return  
PERMNO                       
10001  0            0.00000  
       1                NaN  
       2                NaN  
       3           -0.05311  
       4                NaN  
...                     ...  
93316  50923819         NaN  
       50923820         NaN  
       50923821         NaN  
       50923822         NaN  
       50923823         NaN  

[17223740 rows x 12 columns]